In [0]:
import torch
from torch import nn
import  numpy as np
import random
import torch.utils.data as Data

# 訓練基本設定

In [0]:

TRAINING_SIZE = 900000
TIME_STEP = 7
INPUT_SIZE = 12
TEST_SIZE = 190
BATCH_SIZE = 100
LR = 0.001
answer=[]
input=[]
test_x,test_y = [],[]
EPOCH=5

# 產生訓練及測試數據

In [0]:
for ind in range(TRAINING_SIZE+TEST_SIZE):
    data1 = random.randint(0, 999)
    data2 = random.randint(0, 999)
    a=str(data1+data2)
    i=str(str(data1)+'+'+str(data2))
    if ind < TRAINING_SIZE:
        answer.append(a)
        input.append(i)
    else:
        test_x.append(i)
        test_y.append(int(a))

# 網路架構

In [0]:
rnn = torch.nn.Sequential(
    torch.nn.Linear(84, 70),
    torch.nn.ReLU(),
    torch.nn.Linear(70, 90),
    torch.nn.ReLU(),
    torch.nn.Linear(90, 50),
    torch.nn.ReLU(),
    torch.nn.Linear(50, 48),
)

# 轉換Data(Feature Engineering)

In [0]:

def datatransform(ort,SIZE,num):
    x=np.zeros((SIZE,num,12),dtype=int)
    x=x.tolist()
    for step in range(SIZE):
        for i in range(num):
            if i >= len( ort[step]):
                break
            elif ort[step][i] == '+':
                x[step][i][10] = 1
            elif ort[step][i] == '-':
                x[step][i][11] = 1
            else:
                #print(i)
                x[step][i][int(ort[step][i])] = 1
    return x

# 將訓練資料形成數據集

In [0]:
x = datatransform(input, TRAINING_SIZE,7)
y = datatransform(answer, TRAINING_SIZE,4)
x = torch.Tensor(x).cuda()
y = torch.Tensor(y).cuda()
torch_dataset = Data.TensorDataset(x,y)
train_loader = torch.utils.data.DataLoader(dataset=torch_dataset, batch_size=BATCH_SIZE, shuffle=True)

# 轉換測試數據

In [0]:
test_xx = datatransform(test_x,TEST_SIZE,7)
test_x = torch.Tensor(test_xx).cuda()


# 網路基本設定

In [0]:
rnn.cuda()
optimizer = torch.optim.Adam(rnn.parameters(), lr=LR)   # optimize all cnn parameters
loss_func = nn.MSELoss()

# 訓練20次

In [0]:
for epoch in range(EPOCH):
    for step, (x, y) in enumerate(train_loader):
        x=x.view(-1,84)
        y=y.view(-1,48)
        prediction  = rnn(x)
        loss = loss_func(prediction, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if step % 500 == 0:
            test_x = test_x.view(-1, 84)
            test_output = rnn(test_x)            # (samples, time_step, input_size)
            print('Epoch: ', epoch, '| train loss: %.4f' % loss.data.cpu().numpy())

Epoch:  0 | train loss: 0.0806
Epoch:  0 | train loss: 0.0518
Epoch:  0 | train loss: 0.0456
Epoch:  0 | train loss: 0.0425
Epoch:  0 | train loss: 0.0378
Epoch:  0 | train loss: 0.0376
Epoch:  0 | train loss: 0.0369
Epoch:  0 | train loss: 0.0358
Epoch:  0 | train loss: 0.0341
Epoch:  0 | train loss: 0.0336
Epoch:  0 | train loss: 0.0321
Epoch:  0 | train loss: 0.0332
Epoch:  0 | train loss: 0.0317
Epoch:  0 | train loss: 0.0316
Epoch:  0 | train loss: 0.0306
Epoch:  0 | train loss: 0.0318
Epoch:  0 | train loss: 0.0308
Epoch:  0 | train loss: 0.0289
Epoch:  1 | train loss: 0.0294
Epoch:  1 | train loss: 0.0283
Epoch:  1 | train loss: 0.0270
Epoch:  1 | train loss: 0.0282
Epoch:  1 | train loss: 0.0285
Epoch:  1 | train loss: 0.0257
Epoch:  1 | train loss: 0.0253
Epoch:  1 | train loss: 0.0247
Epoch:  1 | train loss: 0.0246
Epoch:  1 | train loss: 0.0236
Epoch:  1 | train loss: 0.0221
Epoch:  1 | train loss: 0.0235
Epoch:  1 | train loss: 0.0225
Epoch:  1 | train loss: 0.0220
Epoch:  

# 生成展示數據

In [0]:
mydata=[]
myans=[]
for ind in range(30):
    data1 = random.randint(0, 999)
    data2 = random.randint(0, 999)
    a=str(data1+data2)
    i=str(str(data1)+'+'+str(data2))
    myans.append(a)
    mydata.append(i)

# 轉換展示數據

In [0]:
mydata_tran=datatransform(mydata,30,7)
mydata_tran=torch.Tensor(mydata_tran).cuda()

In [0]:
mydata_tran= mydata_tran.view(-1, 84)
mydata_output = rnn(mydata_tran)
mydata_output=mydata_output.cpu().data.numpy()
data = mydata_output.reshape(30,4,12).tolist()

# 訓練結果展示

In [0]:
for i in range(30):
  p=' '
  for j in range(4):
    a=str(data[i][j].index(max(data[i][j])))
    p=p+a
  print('data=',mydata[i],'pre=',p,'ans=',myans[i])


data= 926+427 pre=  1353 ans= 1353
data= 864+375 pre=  1239 ans= 1239
data= 31+675 pre=  7065 ans= 706
data= 298+991 pre=  1289 ans= 1289
data= 646+5 pre=  6731 ans= 651
data= 923+331 pre=  1254 ans= 1254
data= 401+471 pre=  8727 ans= 872
data= 505+324 pre=  8297 ans= 829
data= 971+198 pre=  1169 ans= 1169
data= 870+150 pre=  1020 ans= 1020
data= 671+440 pre=  1111 ans= 1111
data= 154+261 pre=  4151 ans= 415
data= 920+637 pre=  1557 ans= 1557
data= 647+338 pre=  9951 ans= 985
data= 487+967 pre=  1454 ans= 1454
data= 27+739 pre=  7667 ans= 766
data= 973+536 pre=  1509 ans= 1509
data= 96+101 pre=  1977 ans= 197
data= 556+23 pre=  5798 ans= 579
data= 161+993 pre=  1154 ans= 1154
data= 33+563 pre=  5964 ans= 596
data= 766+604 pre=  1370 ans= 1370
data= 771+282 pre=  1053 ans= 1053
data= 52+52 pre=  1943 ans= 104
data= 439+343 pre=  7928 ans= 782
data= 333+550 pre=  8833 ans= 883
data= 491+405 pre=  9968 ans= 896
data= 650+392 pre=  1042 ans= 1042
data= 845+240 pre=  1085 ans= 1085
data= 14